# Notes for Insception Model

Training is going smoothly loss value is at 0.7 and at step ~4000

Might me a good idea to increse the frequency of decay rate currently it is every 30 epochs calculated like so


$$
batches \; per \; epoch = \frac{epoch \; size}{batch \; size}
$$

$$
steps\;before\;decay = (batches\;per\;epoch)(x) \; where \, x = the\; number\;of\;epochs\;before\;decay
$$

### Update

Try upping the batchsize from 32 to 64 or 128

Try inceasing memory factor for faster epochs

### Update

Training had to be restarted, putting memory factor to 8 and batch size to 128

Lowering batch size to 64, because running out of memory at 128

### Update 

Training started `100` step in, but batches take far longer. The learning rate looks good, however training will take far longer, I will leave it overnight and check the progress tomorrow.

I had to restart the training a few times, once because the events writer messed up because of missing files and the other times to up the memory factory to `10` and the the thread & read count to `8`. 

Having terrible little mistakes forgot to update the script from deleting the train folder on restart, luckily I caught this early and was able to fix it before wasting too much time.

### Update step `100`

Loss: 1.18  
Proccessed Images: Good  
Learn rate: .001  
queue: .984  

Looks like training is off to a good start, will check back in the morning.

### Update step 1800 

x entropy loss: 0.7025  
Learn rate: 1e-2  
Total loss: 1.249  
Queue: .9453  

The metrics look good Queue is full, loss is steadily decreaseing sparsity of the layers looks good as well. I think the previous adustjemnts helped, the only concerning issue is the examples/sec has dropped dramatically, there must be a bottle neck somewhere I will have to investigate, my suspicion is that image preproccessing is the cause.

### Update Bottleneck

I have been monitoring the stats and performance of the GPU, it seems the GPU is only at utilization in small spurts, there must be a bottle neck at the CPU or in the transfer of information from CPU --> GPU. I can show that it is not the GPUs memory that is the bottle neck, so that leaves the CPU or the main memory as the bottleneck.  

I will need to look futher into my suspision, but it seems that there are periods where a single core is maxed out for some time until all cores kick in and GPU kicks in, this leads me to beleive there is some `hot` code that is on a single python thread.

### Update step 2800

Entropy loss: .69
Total loss: 1.23
Queue: Good
Learn rate: 1e-2

Training is going fine, but learning is slowing, sparsity is going up, so that is a good sign.

### Update bottleneck

I Updated the training pipline to leave out bounding box manipulation and I belive I found the source of the bottle neck. There was a condition to check the number of readers and set the to the number specified in the flags, however this check was always skipped because, the default was set to 1. I believe the bottle neck is the file reading. I will test this out once the script gets to step 5000 and saves.

### Update step 5000 

Stoped training at save to update put in new script, will resume training probably tomorrow. Loss is .61 and the learn rate dropped, everything looks good so far. Implemented new preprocessing and is 270 steps in and is running well. I believe I have eliminated the bottle neck, lets see if it holds.

Tot Loss : 1
xLoss : .61

### Update step 7400 

Going to force all data onto GPU to see if I can get a speed up, so far training is fine. Loss has leveld off.  
The GPU only approch is actually slower thean CPU + GPU, but I still haven't figured out why there is a slow down.

### Update 

- Changed the saving and summary writing to use global step.  
- Changed the number of readers and threads to 4 to use all 8 cores.  
- Might move whole compute to GPU for faster trining? Need to test this out.  
- Made the file name queue size bigger 16 --> 32.

I need to approch chaning of hyperparams more scientifically, ie only changing one then testing the result. 

The next thing I will change is the file name size to see if that changes performance.

Also put in proframces summary ie examples persecond or time per batch or both.

### Update

The bottle neck bug magically resolved it self after I ran a Tensorflow build??? I submitted an issue to try and figure out what is going on, obviously I'm missing something.

### Update

Found a tretrain script in the tensorflow repository, going to test it out. It appears to be a one stop script, by that I mean it just requires a data dir and it creates all files and does all nessecary preprocessing, then trains the model with the output.